In [34]:
%%time
# endpoint invocation script
# expects the file test_point.csv to be located in the same diretory

#endpoint_name = 'XGBoostEP-2020-03-15-16-23-35'
#endpoint_name = 'XGBoostEP-2020-03-15-20-37-40'
endpoint_name = 'aparna-ee-1'

import os
import boto3
import re
import json
from sagemaker import get_execution_role
import sagemaker
import time

sess = sagemaker.Session()
region = boto3.Session().region_name
role = get_execution_role()
bucket=sess.default_bucket()
sm_client = boto3.client('sagemaker')
runtime_client = boto3.client('runtime.sagemaker')

file_name = 'test_point.csv' #customize to your test file, will be 'mnist.single.test' if use data above

with open(file_name, 'r') as f:
    payload = f.read().strip()

response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='text/csv', 
                                   Body=payload)
result = response['Body'].read().decode('ascii')
#print('Predicted Class Probabilities: {}.'.format(result))
print(response)

file_name = 'test_point.csv'
with open(file_name, 'r') as f:
    payload = f.read().strip()
    

{'ResponseMetadata': {'RequestId': '3992533a-7599-4823-a194-dc335b775a4b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '3992533a-7599-4823-a194-dc335b775a4b', 'x-amzn-invoked-production-variant': 'Initial', 'date': 'Sun, 29 Mar 2020 19:59:55 GMT', 'content-type': 'text/csv; charset=utf-8', 'content-length': '2'}, 'RetryAttempts': 0}, 'ContentType': 'text/csv; charset=utf-8', 'InvokedProductionVariant': 'Initial', 'Body': <botocore.response.StreamingBody object at 0x7f6d3d328160>}
CPU times: user 192 ms, sys: 11.9 ms, total: 204 ms
Wall time: 380 ms


### Let us scale up the endpoint by sending in the traffic well above 100 invocations per min

In [35]:
N = 8000 #5000
tic=time.time()
for i in range(N):
    response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, ContentType='text/csv', Body=payload)
    if i % (300) == 0:
        invoked_model = response['ResponseMetadata']['HTTPHeaders']['x-amzn-invoked-production-variant']
        invocation_code = response['ResponseMetadata']['HTTPStatusCode']
        toc=time.time()
        sm_client_resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
        status = sm_client_resp['EndpointStatus']
        InstanceCount=sm_client_resp['ProductionVariants'][0]['CurrentInstanceCount']
        print ("model: {}, rtrn code: {}; {} of {} invocations, elapsed time: {:.0f} s, EP status: {}, instance cnt: {}"\
               .format(invoked_model, invocation_code, i, N, toc-tic, status, InstanceCount))
        tic = time.time()
    time.sleep(0.1)

model: Initial, rtrn code: 200; 0 of 6000 invocations, elapsed time: 0 s, EP status: InService, instance cnt: 1
model: Initial, rtrn code: 200; 300 of 6000 invocations, elapsed time: 41 s, EP status: InService, instance cnt: 1
model: Initial, rtrn code: 200; 600 of 6000 invocations, elapsed time: 39 s, EP status: InService, instance cnt: 1
model: Initial, rtrn code: 200; 900 of 6000 invocations, elapsed time: 38 s, EP status: InService, instance cnt: 1
model: Initial, rtrn code: 200; 1200 of 6000 invocations, elapsed time: 39 s, EP status: InService, instance cnt: 1
model: Initial, rtrn code: 200; 1500 of 6000 invocations, elapsed time: 39 s, EP status: InService, instance cnt: 1
model: Initial, rtrn code: 200; 1800 of 6000 invocations, elapsed time: 38 s, EP status: InService, instance cnt: 1
model: Initial, rtrn code: 200; 2100 of 6000 invocations, elapsed time: 38 s, EP status: InService, instance cnt: 1
model: Initial, rtrn code: 200; 2400 of 6000 invocations, elapsed time: 39 s, E

### now let us scale down the endpoint by sending in the traffic well below 100 invocations per min
Please, note that if you send no traffic at all, the endpoint will not scale down.

In [38]:
N = 1200 #5000
tic=time.time()
for i in range(N):
    response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, ContentType='text/csv', Body=payload)
    if i % (30) == 0:
        invoked_model = response['ResponseMetadata']['HTTPHeaders']['x-amzn-invoked-production-variant']
        invocation_code = response['ResponseMetadata']['HTTPStatusCode']
        toc=time.time()
        sm_client_resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
        status = sm_client_resp['EndpointStatus']
        InstanceCount=sm_client_resp['ProductionVariants'][0]['CurrentInstanceCount']
        print ("model: {}, rtrn code: {}; {} of {} invocations, elapsed time: {:.0f} s, EP status: {}, instance count: {}"\
               .format(invoked_model, invocation_code, i, N, toc-tic, status, InstanceCount))
        tic = time.time()
    time.sleep(2)

model: Initial, rtrn code: 200; 0 of 1200 invocations, elapsed time: 0 s, EP status: InService, instance count: 3
model: Initial, rtrn code: 200; 30 of 1200 invocations, elapsed time: 61 s, EP status: InService, instance count: 3
model: Initial, rtrn code: 200; 60 of 1200 invocations, elapsed time: 61 s, EP status: Updating, instance count: 3
model: Initial, rtrn code: 200; 90 of 1200 invocations, elapsed time: 61 s, EP status: Updating, instance count: 2
model: Initial, rtrn code: 200; 120 of 1200 invocations, elapsed time: 61 s, EP status: Updating, instance count: 2
model: Initial, rtrn code: 200; 150 of 1200 invocations, elapsed time: 61 s, EP status: Updating, instance count: 2
model: Initial, rtrn code: 200; 180 of 1200 invocations, elapsed time: 61 s, EP status: InService, instance count: 2
model: Initial, rtrn code: 200; 210 of 1200 invocations, elapsed time: 61 s, EP status: InService, instance count: 2
model: Initial, rtrn code: 200; 240 of 1200 invocations, elapsed time: 61 

KeyboardInterrupt: 

In [37]:
N = 200 #5000
print("started model invocation for {} iterations". format(N))
for i in range(N):
    response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, ContentType='text/csv', Body=payload)
    invoked_model = response['ResponseMetadata']['HTTPHeaders']['x-amzn-invoked-production-variant']
    invocation_code = response['ResponseMetadata']['HTTPStatusCode']
    if (invocation_code != 200):
        print ("error invoking model {} ".format(invocation_code))

started model invocation for 200 iterations
